<a href="https://colab.research.google.com/github/najumattos/Gerador-post-IA/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
%pip -q install google-genai

In [47]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
#variavel de ambiente que pega a APIKEY do segredo
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [48]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [49]:
#Somente com base no conhecimento que o modelo possui
# Pergunta ao Gemini uma informação mais recente que seu conhecimento


from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model = MODEL_ID,
    contents = 'Quando é a próxima imersão IA com Google Gemini da Alura?'
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma "Imersão IA com Google Gemini" agendada no momento. Para saber sobre os próximos cursos e imersões da Alura, incluindo os relacionados a IA e Google Gemini, a melhor forma é verificar diretamente no site oficial da Alura:

*   **Site da Alura:** [https://www.alura.com.br/](https://www.alura.com.br/)

Você pode procurar por cursos e formações na área de Inteligência Artificial ou usar a barra de pesquisa para encontrar algo específico sobre Google Gemini.

In [50]:
#Busca no goole, usa-se o tools (ferramenta) do google_search, porém eu poderia criar um agente que busque por exemplo no google hotels, maps, etc
#Posso ter mais de um agente no mesmo projeto
#Pode ser passado tudo pelo mesmo agente ou em agentes separados (Acho que deve ser o mesmo principio de codigo limpo em relação a metodos())
#Varios agentes especializados, cada um com um prompt para retornar as respostas
###Pergunta: Quais tipos de ferramentas posso utilizar? Posso buscar infos da onde além dessas do google?

# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',

    # Inserir a tool de busca do Google ###############################################
    config = { "tools": [{"google_search": {}}]

    }
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio e são totalmente gratuitas.

In [51]:
#Validação da resposta do google mostrando as fontes das respostas
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini data']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [52]:
#O que é um agente? Cada um tem sua opnião mas de uma forma abstrata um agente cria uma esteira de ações onde só é necessário escrever o código uma vez
#O Agente permite programar para rodar as etapas na ordem correta de maneira organizada e automatizada
#A criação de uma gente é como estrutura as etapas usando alguma ferramenta que simplifique o processo para que não precise ser feito de maneira manual
#Cada agente depende da execução do agente anterior. Cada agente é como se fosse uma etapa
#Para isso que se utiliza um framework, para simoplicar o processo
#
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk



In [53]:
##24:36 imersao Alura
from google.adk.agents import Agent ##abstração de um objeto de agente
from google.adk.runners import Runner ##Orquestrador que roda tudo
from google.adk.sessions import InMemorySessionService #Memoria interna do orquestrador
from google.adk.tools import google_search #tool importada
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
##Para formatar saída
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [54]:
#Essa função será a que controlará os agentes
#A gente manda o agente a ser utilizado pra ca e essa função roda tudo que for necessario
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

#A Função recebe o agente e o prompt como parametros
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [55]:
# Função auxiliar para exibir texto formatado em Markdown no Colab

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [56]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

#Esse primeiro agente busca no google as notícias  sobre acontecimentos do último mês e passa para o segundo agente que é o PLANEJADOR

def agente_buscador(topico, data_de_hoje):
  #esse Agent() está sendo importado no primeiro bloco de código
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",

        #Parâmetro opcional de documentação. É uma descrição para facilitar o entendimento do que esse agente faz quando outra pessoa for usar o código
        description = "Agente Buscador de informações no Google",

        #Chamar a ferramenta de busca (pega do import do google_search)
        tools = [google_search],

        #Calibrar como a IA vai responder
        instruction = """Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do goole (google_search)
        para recuperar as últimas notícias de lançamentos muito relçevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamanetos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim e pode ser substituído por outro que tenha mais.
        Esses lançamentos devem ser atuais, de no máximo um mês antes da data de hoje."""
        )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente atraves da função call_agent() criada acima
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados


In [57]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################

#Esse segundo agente recebe as notícias que o primeiro agente buscou
#Ele busca no google mais detalhes sobre essas notícias de acordo com o prompt passado
#Depois o planejamento é passado para o terceiro agente, que é o REDATOR

def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",

        # Inserir as instruções do Agente Planejador #################################################
        tools=[google_search],
        description="Agente Planejador que detalha as informações recebidas do agente buscador",
        instruction="""Você é um planejador de conteúdo, especialista em redes sociais.
        Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google(google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o google_search para para encontrar mais informações sobre os temas e se aprofundar.
        Ao final você irá escolher o tema mais relevante, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [63]:
######################################
# --- Agente 3: Redator do Post --- #
######################################

#Esse terceiro agente recebe o planejamento do segundo agente e cria o post de acordo com o prompt passado
#Depois o post gerado é passado para o quarto agente, o REVISOR

def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        description="Agente Redator de posts engajadores para Instagram",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """
            )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [66]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################

#Esse quarto agente recebe o texto do terceiro, faz uma revisão de acordo com o prompt passado para criar o post
#Depois de revisado, o post criado é enviado para o leitor

def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
          description="Agente revisor de post para redes sociais.",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [68]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print("Você não digitou o tópico")
else:
     print(f"Maravilha!! Vamos então criar o post sobre novidades em {topico}")

     lancamentos_buscados = agente_buscador(topico, data_de_hoje)
     print("\n -- Resultado do Agente Um (Buscador) --\n")
     display(to_markdown(lancamentos_buscados))
     print("----------------------------------------------------------------------------------------------------")

     plano_de_post = agente_planejador(topico, lancamentos_buscados)
     print("\n -- Resultado do Agente Dois (Planejador) --\n")
     display(to_markdown(plano_de_post))
     print("----------------------------------------------------------------------------------------------------")

     rascunho_gerado = agente_redator(topico, plano_de_post)
     print("\n -- Resultado do Agente Três (Redator) --\n")
     display(to_markdown(rascunho_gerado))
     print("----------------------------------------------------------------------------------------------------")

     texto_revisado = agente_revisor(topico,rascunho_gerado)
     print("\n -- Resultado do Agente Quatro (Revisor) --\n")
     display(to_markdown(texto_revisado))
     print("----------------------------------------------------------------------------------------------------")







🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Maravilha!! Vamos então criar o post sobre novidades em agentes de IA

 -- Resultado do Agente Um (Buscador) --



> Para encontrar os lançamentos mais relevantes e recentes sobre agentes de IA, vou usar o Google para pesquisar notícias e artigos sobre o tema. Procurarei por lançamentos que geraram entusiasmo e cobertura notável na mídia, focando nos últimos meses.
> 
> 
> Com base nas notícias mais recentes sobre agentes de IA, aqui estão alguns lançamentos e tendências relevantes, com foco nos últimos meses (considerando a data de hoje, 18 de maio de 2025):
> 
> 1.  **Adoção e Integração Empresarial de Agentes de IA:**
>     *   Muitas empresas estão implementando agentes de IA para melhorar a tomada de decisões e criar operações mais inteligentes e ágeis. A Forrester prevê que 50% das empresas adotarão esses recursos até o final de 2025 \[1].
>     *   Empresas como IBM, Anthropic, Amazon, Google e OpenAI estão lançando atualizações e ferramentas para integrar agentes de IA em diversos setores, desde RH e vendas até compras \[3].
>     *   A IBM anunciou atualizações no watsonx Orchestrate, facilitando a integração e customização de agentes com mais de 80 aplicações empresariais \[3].
> 
> 2.  **Agentes de IA como Força de Trabalho Digital:**
>     *   A Nvidia revelou o AI-Q Blueprint, visando transformar sistemas de agentes de IA em forças de trabalho digitais \[10].
>     *   A B3 (Bolsa de Valores Brasileira) está adotando agentes de IA para revolucionar suas operações em 2025 \[12].
> 
> 3.  **Capacidades Aprimoradas e Autonomia:**
>     *   Agentes de IA estão evoluindo para além da IA generativa, focando na ação e execução de tarefas de forma autônoma e autodirigida, sem instruções explícitas \[2].
>     *   Eles combinam maior poder de síntese de dados e capacidades de tomada de decisão, executando tarefas a partir de listas de etapas de forma independente \[2].
>     *   Modelos como o "computer use" da Anthropic (uma extensão do Claude) permitem que agentes usem computadores de maneira semelhante a humanos, manipulando o cursor, clicando em botões e digitando texto \[6].
> 
> 4.  **Integração com Ferramentas e Plataformas Existentes:**
>     *   Agentes de IA estão se tornando mais acessíveis e integrados aos ambientes onde as pessoas trabalham, com a expectativa de que os custos de API caiam drasticamente \[4].
>     *   A OpenAI integrou o ChatGPT com o WhatsApp, permitindo que usuários façam perguntas diretamente ao assistente e recebam respostas com base em buscas em tempo real na internet \[4].
> 
> 5.  **Desafios e Considerações:**
>     *   Há um debate sobre o que realmente constitui um "agente de IA", com alguns especialistas argumentando que muitas soluções rotuladas como tal são apenas respostas prontas disfarçadas de assistentes digitais \[11].
>     *   Questões como privacidade de dados, regulamentação e impacto no mercado de trabalho exigem um debate amplo para garantir que o avanço tecnológico seja inclusivo e sustentável \[5].
>     *   A Microsoft enfatiza que a supervisão humana continuará sendo crucial na evolução dos agentes de IA para garantir a segurança e a responsabilidade \[16].
> 
> Esses lançamentos e tendências destacam a crescente importância dos agentes de IA em diversos setores, com um foco em automação, eficiência e capacidades aprimoradas. No entanto, também ressaltam a necessidade de abordar os desafios e considerações éticas associadas a essa tecnologia em rápida evolução.
> 


----------------------------------------------------------------------------------------------------

 -- Resultado do Agente Dois (Planejador) --



> Com base nos lançamentos e tendências sobre agentes de IA, podemos criar posts focados nos seguintes pontos:
> 
> **1. Adoção Empresarial:**
> 
> *   **Assuntos para o Post:**
>     *   Crescente adoção de agentes de IA em empresas.
>     *   Como agentes de IA estão melhorando a tomada de decisões e agilidade.
>     *   Exemplos de empresas (IBM, Anthropic, Amazon, Google, OpenAI) e suas soluções.
>     *   Detalhes sobre atualizações como o watsonx Orchestrate da IBM.
> *   **Possível Título:** "Agentes de IA: A Revolução na Tomada de Decisões Empresariais em 2025"
> 
> **2. Agentes de IA como Força de Trabalho Digital:**
> 
> *   **Assuntos para o Post:**
>     *   Conceito de agentes de IA como força de trabalho digital.
>     *   AI-Q Blueprint da Nvidia e seu impacto.
>     *   Adoção de agentes de IA pela B3 (Bolsa de Valores Brasileira).
>     *   Exemplos de tarefas que esses agentes podem realizar.
> *   **Possível Título:** "A Força de Trabalho do Futuro: Como Agentes de IA Estão Transformando as Empresas"
> 
> **3. Capacidades Aprimoradas e Autonomia:**
> 
> *   **Assuntos para o Post:**
>     *   Evolução dos agentes de IA além da IA generativa.
>     *   Foco na autonomia e execução de tarefas sem instruções explícitas.
>     *   Combinação de síntese de dados e tomada de decisão.
>     *   Exemplos como o modelo "computer use" da Anthropic (Claude).
> *   **Possível Título:** "Agentes de IA Autônomos: A Próxima Fronteira da Inteligência Artificial"
> 
> **4. Integração e Acessibilidade:**
> 
> *   **Assuntos para o Post:**
>     *   Facilidade de integração de agentes de IA com ferramentas existentes.
>     *   Expectativa de queda nos custos de API.
>     *   Integração do ChatGPT com o WhatsApp como exemplo de acessibilidade.
>     *   Como a integração facilita o uso diário dos agentes de IA.
> *   **Possível Título:** "Agentes de IA ao Seu Alcance: Integração e Acessibilidade para Todos"
> 
> **5. Desafios e Considerações Éticas:**
> 
> *   **Assuntos para o Post:**
>     *   Debate sobre o que realmente define um "agente de IA".
>     *   Questões de privacidade de dados e regulamentação.
>     *   Impacto no mercado de trabalho.
>     *   Importância da supervisão humana (ênfase da Microsoft).
> *   **Possível Título:** "Agentes de IA: Navegando pelos Desafios Éticos e Garantindo um Futuro Inclusivo"
> 
> **Plano Detalhado para o Tópico Mais Relevante: Capacidades Aprimoradas e Autonomia**
> 
> Este tópico é o mais interessante pois aborda a evolução dos agentes de IA para além das capacidades generativas, focando na autonomia e ação.
> 
> *   **Título:** Agentes de IA Autônomos: A Próxima Fronteira da Inteligência Artificial
> 
> *   **Assuntos a serem Abordados no Post:**
>     1.  **Introdução:**
>         *   O que são agentes de IA e por que eles são importantes.
>         *   A rápida evolução dos agentes de IA nos últimos anos.
>     2.  **A Evolução Além da IA Generativa:**
>         *   Explicação de como os agentes de IA estão indo além da simples geração de texto ou imagens.
>         *   Foco na capacidade de agir e executar tarefas.
>     3.  **Autonomia e Execução de Tarefas:**
>         *   Como os agentes de IA podem operar sem instruções explícitas.
>         *   Exemplos de tarefas que eles podem realizar de forma autônoma.
>     4.  **Capacidades Aprimoradas:**
>         *   Discussão sobre a combinação de síntese de dados e tomada de decisão.
>         *   Como os agentes de IA analisam grandes quantidades de dados para tomar decisões informadas.
>     5.  **Exemplo Prático: Modelo "Computer Use" da Anthropic:**
>         *   Detalhes sobre o modelo "computer use" e como ele permite que agentes usem computadores de maneira semelhante a humanos.
>         *   Exemplos de ações que o agente pode realizar (manipular o cursor, clicar em botões, digitar texto).
>     6.  **Impacto e Aplicações Futuras:**
>         *   Como a autonomia dos agentes de IA pode transformar diversos setores.
>         *   Exemplos de aplicações em áreas como saúde, finanças e manufatura.
>     7.  **Desafios e Considerações:**
>         *   Breve discussão sobre os desafios e considerações éticas relacionados à autonomia dos agentes de IA.
>         *   Importância de garantir que esses agentes sejam usados de forma responsável e segura.
>     8.  **Conclusão:**
>         *   Reafirmação da importância dos agentes de IA autônomos e seu potencial para o futuro.
>         *   Chamada para ação (ex: acompanhar as novidades, explorar as aplicações).
> 
> Com este plano, podemos criar um post informativo e envolvente que destaque a próxima fronteira da inteligência artificial: os agentes de IA autônomos.
> 


----------------------------------------------------------------------------------------------------

 -- Resultado do Agente Três (Redator) --



> 🚀🤖 **Agentes de IA Autônomos: A Próxima Fronteira da Inteligência Artificial!** 🤖🚀
> 
> Já ouviu falar em agentes de IA? Eles são a evolução da inteligência artificial que vai além da simples geração de texto e imagens! 🤯
> 
> Nos últimos anos, eles têm evoluído rapidamente, ganhando a capacidade de **agir** e **executar tarefas** de forma autônoma, sem precisar de instruções a cada passo. 😮
> 
> ✨ **O que eles fazem?**
> Imagine um agente de IA que analisa dados, toma decisões e executa ações como um humano faria, mas com a velocidade e a precisão de uma máquina. É isso que eles fazem!
> 
> 💻 **Exemplo prático:**
> A Anthropic criou um modelo chamado "computer use" que permite que seus agentes usem computadores como nós: movem o cursor, clicam em botões e digitam textos! 🤯
> 
> 💼 **Onde isso impacta?**
> Saúde, finanças, manufatura... A autonomia dos agentes de IA tem o poder de transformar diversos setores!
> 
> 🤔 **E os desafios?**
> Claro, existem desafios e considerações éticas importantes, mas o potencial é incrível.
> 
> Ficou curioso? 🤔 Acompanhe as novidades e explore as aplicações dos agentes de IA. O futuro da tecnologia já começou! 😉
> 
> #InteligenciaArtificial #AgentesDeIA #Inovação #Alura
> 


----------------------------------------------------------------------------------------------------

 -- Resultado do Agente Quatro (Revisor) --



> O rascunho está bom, mas podemos deixá-lo ainda mais interessante para o público do Instagram. Aqui estão algumas sugestões:
> 
> *   **Emojis:** Use emojis de forma mais estratégica para quebrar o texto e destacar informações importantes. Por exemplo, ao invés de usar vários emojis no título, use um ou dois que realmente chamem a atenção.
> *   **Linguagem:** Adapte a linguagem para ser mais direta e envolvente, como se estivesse conversando com um amigo. Evite termos muito técnicos ou jargões que possam confundir o público.
> *   **Chamada para ação (CTA):** Inclua uma chamada para ação mais clara, incentivando os seguidores a interagir.
> 
> **Sugestão de Rascunho Revisado:**
> 
> 🚀🤖 **Agentes de IA: Prontos para Dominar o Mundo (do Trabalho)?** 🤖
> 
> Já imaginou uma IA que faz tudo sozinha, sem você ter que dar cada passo? 🤯 É disso que se tratam os agentes de IA!
> 
> Eles aprenderam a **agir** e **executar tarefas** sozinhos. Tipo, sem precisar de você mandando! 😮
> 
> **Na prática:**
> 
> Eles analisam dados, tomam decisões e fazem acontecer! Tipo um humano, só que 1000x mais rápido. 💻
> 
> A Anthropic criou um agente que usa o computador como a gente: mexe o mouse, clica e digita! Surreal, né? 🤯
> 
> **Onde isso muda tudo?**
> 
> Na saúde, nas finanças, nas fábricas... Eles vão dar um jeito de otimizar tudo! 💼
> 
> **E os desafios?**
> 
> Ainda estamos entendendo como controlar essa galera, mas o potencial é GIGANTE. 🤔
> 
> Curtiu a ideia? Deixe um comentário com a sua opinião! 👇
> 
> #InteligenciaArtificial #AgentesDeIA #Inovação #Alura #futuro


----------------------------------------------------------------------------------------------------
